In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Use plotly offline for fancy plots
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
# use cufflinks to bind plotly to pandas
import cufflinks as cf 
from os import listdir
# for display control
from IPython.display import display
# Gradient boosting using LightBGM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
# Garbage collection
import gc
gc.enable()
# Lock pseudo-number seed
randSeed = 1
np.random.seed(randSeed)

In [2]:
# Global verbose control
PREVIEW_DATASET = 0

In [3]:
# Utility function to transform all catagorical fields using one hot ending
def oneHotEncoding(df):
    # Get list categorical features
    catFeatures = [col for col in df.columns if df[col].dtype == 'object']
    # Convert to one hot encoding
    ohe = pd.get_dummies(df, columns=catFeatures)
    return ohe

# Utility function to add descriptive statistics as secondary fields in the dataframe
def addStatsFields(df, field):
    df['MEAN_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .mean()[field]
    )
    df['MEDIAN_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .median()[field]
    )
    df['MAX_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .max()[field]
    )
    df['MIN_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .min()[field]
    )
    df['SUM_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .sum()[field]
    )
    df['VAR_' + field] = (
        df[['SK_ID_CURR', field]]
            .groupby('SK_ID_CURR')
            .var()[field]
    )
    return df

In [4]:
# Load all data
dataFiles = listdir("../data/home-credit-default-risk/")
for filename in dataFiles:
    print(f'loading {filename} ...')
    if '.csv.zip' in filename:
        # compressed data file
        locals()[filename.rstrip('.csv.zip')] = pd.read_csv(
            f'../data/home-credit-default-risk/{filename}',
            compression='zip', 
            header=0, 
            sep=',', 
            quotechar='"'
        )

dataTrain = oneHotEncoding(application_train)
dataTest = oneHotEncoding(application_test)
print('done')

loading application_train.csv.zip ...
loading previous_application.csv.zip ...
loading lowpass_submission_v1.csv ...
loading bureau.csv.zip ...
loading credit_card_balance.csv.zip ...
loading application_test.csv.zip ...
loading installments_payments.csv.zip ...
loading bureau_balance.csv.zip ...
loading POS_CASH_balance.csv.zip ...
loading HomeCredit_columns_description.csv ...
loading sample_submission.csv.zip ...
done


In [5]:
## Preprocess bureau datasets
if PREVIEW_DATASET:
    print('Raw bureau_balance dataset')
    display(bureau_balance.head(5))

# Count by status
bureauBalance = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts(normalize = False)
# Pivot into table of status values
bureauBalance = bureauBalance.unstack('STATUS')
# Add months balance data as new fileds
bureauBalance['MONTHS_COUNT'] = (
    bureau_balance
        .groupby('SK_ID_BUREAU') 
        .MONTHS_BALANCE          
        .size()
)
bureauBalance['MONTHS_MAX'] = (
    bureau_balance
        .groupby('SK_ID_BUREAU')
        .MONTHS_BALANCE
        .max()
)
bureauBalance['MONTHS_MIN'] = (
    bureau_balance
        .groupby('SK_ID_BUREAU')
        .MONTHS_BALANCE
        .min()
)
if PREVIEW_DATASET:
    print('Formatted')
    display(bureauBalance.head(5))

# Finally, merge the two bureau table together 
bureauData = bureau.join(bureauBalance, how='left', on='SK_ID_BUREAU')

# Transform features
bureauData = oneHotEncoding(bureauData).groupby('SK_ID_CURR').mean()
if PREVIEW_DATASET:
    print('Merged and transformed')
    display(bureauData.head(5))

# Merge features into main training dataset
featureCntBefore = len(dataTrain.keys()) - 1
dataTrain = dataTrain.merge(
    right = bureauData,
    how = 'left',
    on = 'SK_ID_CURR'
)
dataTest = dataTest.merge(
    right = bureauData,
    how = 'left',
    on = 'SK_ID_CURR'
)
featureCntAfter = len(dataTrain.keys()) - 1
newFeatureCnt = featureCntAfter - featureCntBefore
# Show stats
print(f'bureau datasets processed, {newFeatureCnt} new features added')

# Remove temporary variables and clean up memory
del bureauBalance
del bureauData
del bureau_balance
del bureau
gc.collect();

bureau datasets processed, 47 new features added


In [6]:
## Preproces previous_application
# Transform with one hot encoding
prevApplication = oneHotEncoding(previous_application)

# Compute number of previous applications by counting SK_ID_PREV
prevApplicationCnt = (
    prevApplication[['SK_ID_CURR', 'SK_ID_PREV']]
        .groupby('SK_ID_CURR')
        .count()
)
# Remove the SK_ID_PREV feature because the average of it is meaningless
del prevApplication['SK_ID_PREV']
# Group by mean
prevApplication = prevApplication.groupby('SK_ID_CURR').mean()
# Add back the number of previous applications
prevApplication['PREV_APPLICATION_CNT'] = prevApplicationCnt['SK_ID_PREV']

# Display merged dataset
if PREVIEW_DATASET:
    print('Merged dataset')
    display(prevApplication.head(5))

# Merge features into main training dataset
featureCntBefore = len(dataTrain.keys()) - 1
dataTrain = dataTrain.merge(
    right = prevApplication,
    how = 'left',
    on = 'SK_ID_CURR'
)
dataTest = dataTest.merge(
    right = prevApplication,
    how = 'left',
    on = 'SK_ID_CURR'
)
featureCntAfter = len(dataTrain.keys()) - 1
newFeatureCnt = featureCntAfter - featureCntBefore
# Show stats
print(f'previous_application datasets processed, {newFeatureCnt} new features added')

# Remove temporary variables and clean up memory
del prevApplicationCnt
del prevApplication
del previous_application
gc.collect();

previous_application datasets processed, 163 new features added


In [7]:
## Preproces POS_CASH_balance
# Transform with one hot encoding
posCashBal = oneHotEncoding(POS_CASH_balance)

# Add some secondary features
posCashBal = addStatsFields(posCashBal, 'SK_DPD')
posCashBal = addStatsFields(posCashBal, 'SK_DPD_DEF')
posCashBal = addStatsFields(posCashBal, 'CNT_INSTALMENT_FUTURE')
posCashBal = addStatsFields(posCashBal, 'CNT_INSTALMENT')

# Group by UserID
del posCashBal['SK_ID_PREV']
posCashBal = posCashBal.groupby('SK_ID_CURR').mean()

# Display merged dataset
if PREVIEW_DATASET:
    print('Processed dataset')
    display(posCashBal.head())
    
# Merge features into main training dataset
featureCntBefore = len(dataTrain.keys()) - 1
dataTrain = dataTrain.merge(
    right = posCashBal,
    how = 'left',
    on = 'SK_ID_CURR'
)
dataTest = dataTest.merge(
    right = posCashBal,
    how = 'left',
    on = 'SK_ID_CURR'
)
featureCntAfter = len(dataTrain.keys()) - 1
newFeatureCnt = featureCntAfter - featureCntBefore
# Show stats
print(f'POS_CASH_balance dataset processed, {newFeatureCnt} new features added')

# Remove temporary variables
del posCashBal
del POS_CASH_balance
gc.collect();    

POS_CASH_balance dataset processed, 38 new features added


In [8]:
## Preproces credit_card_balance
# Transform with one hot encoding
# Rename fields that have already used in POS_CASH_balance
creditCardBal = credit_card_balance.rename(
    index = str, 
    columns = {
        'NAME_CONTRACT_STATUS' : 'NAME_CONTRACT_STATUS_CREDIT',
        'SK_DPD' : 'SK_DPD_CREDIT',
        'SK_DPD_DEF' : 'SK_DPD_DEFCREDIT'
    }
)
creditCardBal = oneHotEncoding(creditCardBal)

# Add some secondary features
creditCardBal = addStatsFields(creditCardBal, 'AMT_BALANCE')
creditCardBal = addStatsFields(creditCardBal, 'AMT_CREDIT_LIMIT_ACTUAL')
creditCardBal = addStatsFields(creditCardBal, 'AMT_DRAWINGS_ATM_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_DRAWINGS_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_DRAWINGS_OTHER_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_DRAWINGS_POS_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_INST_MIN_REGULARITY')
creditCardBal = addStatsFields(creditCardBal, 'AMT_PAYMENT_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_PAYMENT_TOTAL_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'AMT_RECEIVABLE_PRINCIPAL')
creditCardBal = addStatsFields(creditCardBal, 'AMT_RECIVABLE')
creditCardBal = addStatsFields(creditCardBal, 'AMT_TOTAL_RECEIVABLE')
creditCardBal = addStatsFields(creditCardBal, 'CNT_DRAWINGS_ATM_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'CNT_DRAWINGS_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'CNT_DRAWINGS_OTHER_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'CNT_DRAWINGS_POS_CURRENT')
creditCardBal = addStatsFields(creditCardBal, 'CNT_INSTALMENT_MATURE_CUM')

# Group by UserID
del creditCardBal['SK_ID_PREV']
creditCardBal = creditCardBal.groupby('SK_ID_CURR').mean()

# Display merged dataset
if PREVIEW_DATASET:
    print('Processed dataset')
    display(creditCardBal.head())
    
# Merge features into main training dataset
featureCntBefore = len(dataTrain.keys()) - 1
dataTrain = dataTrain.merge(
    right = creditCardBal,
    how = 'left',
    on = 'SK_ID_CURR'
)
dataTest = dataTest.merge(
    right = creditCardBal,
    how = 'left',
    on = 'SK_ID_CURR'
)
featureCntAfter = len(dataTrain.keys()) - 1
newFeatureCnt = featureCntAfter - featureCntBefore
# Show stats
print(f'credit_card_balance dataset processed, {newFeatureCnt} new features added')

# Remove temporary variables
del creditCardBal
del credit_card_balance
gc.collect();

POS_CASH_balance dataset processed, 129 new features added


In [49]:
colDictMean = {}
colDictMin = {}
colDictMax = {}
colDictMedian = {}
colDictVar = {}
for col in installments_payment.columns:
    if col not in ['SK_ID_CURR', 'SK_ID_PREV']:
        colDictMean[col] = 'MEAN_' + col
        colDictMin[col] = 'MIN_' + col
        colDictMax[col] = 'MAX_' + col
        colDictMedian[col] = 'MEDIAN_' + col
        colDictVar[col] = 'VAR_' + col

# Add mean
installPayment = installments_payment.groupby('SK_ID_CURR').mean()
del installPayment['SK_ID_PREV']
installPayment = installPayment.rename(
    columns = colDictMean
)
# Add min
installPaymentMin = installments_payment.groupby('SK_ID_CURR').min()
del installPaymentMin['SK_ID_PREV']
installPaymentMin = installPaymentMin.rename(
    columns = colDictMin
)
installPayment = installPayment.merge(
    right = installPaymentMin,
    how = 'left',
    on = 'SK_ID_CURR'
)
del installPaymentMin
# Add max
installPaymentMax = installments_payment.groupby('SK_ID_CURR').max()
del installPaymentMax['SK_ID_PREV']
installPaymentMax = installPaymentMax.rename(
    columns = colDictMax
)
installPayment = installPayment.merge(
    right = installPaymentMax,
    how = 'left',
    on = 'SK_ID_CURR'
)
del installPaymentMax
# Add median
installPaymentMedian = installments_payment.groupby('SK_ID_CURR').median()
del installPaymentMedian['SK_ID_PREV']
installPaymentMedian = installPaymentMedian.rename(
    columns = colDictMedian
)
installPayment = installPayment.merge(
    right = installPaymentMedian,
    how = 'left',
    on = 'SK_ID_CURR'
)
del installPaymentMedian
# Add variance
installPaymentVar = installments_payment.groupby('SK_ID_CURR').var()
del installPaymentVar['SK_ID_PREV']
installPaymentVar = installPaymentVar.rename(
    columns = colDictVar
)
installPayment = installPayment.merge(
    right = installPaymentVar,
    how = 'left',
    on = 'SK_ID_CURR'
)
del installPaymentVar

# Merge features into main training dataset
featureCntBefore = len(dataTrain.keys()) - 1
dataTrain = dataTrain.merge(
    right = installPayment,
    how = 'left',
    on = 'SK_ID_CURR'
)
dataTest = dataTest.merge(
    right = installPayment,
    how = 'left',
    on = 'SK_ID_CURR'
)
featureCntAfter = len(dataTrain.keys()) - 1
newFeatureCnt = featureCntAfter - featureCntBefore
# Show stats
print(f'installments_payment dataset processed, {newFeatureCnt} new features added')

# Remove temporary variables
del installPayment
del installments_payment
gc.collect();

installments_payment dataset processed, 30 new features added


In [11]:
## Preproces POS_CASH_balance
# Transform with one hot encoding
#posCashBal = oneHotEncoding(POS_CASH_balance)
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
#display(POS_CASH_balance.head(5))
#oneHotEncoding(POS_CASH_balance).head(5)
#POS_CASH_balance['NAME_CONTRACT_STATUS'].astype(str)
posCashBal = POS_CASH_balance
le = LabelEncoder()
posCashBal.NAME_CONTRACT_STATUS = le.fit_transform(posCashBal['NAME_CONTRACT_STATUS'].astype(str))
posCashBal['NUM_UNIQUE_STATUS'] = (
    posCashBal[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']]
        .groupby('SK_ID_CURR')
        .nunique()
        .NAME_CONTRACT_STATUS
)
posCashBal['NUM_MAX_STATUS'] = (
    posCashBal[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']]
        .groupby('SK_ID_CURR')
        .max()
        .NAME_CONTRACT_STATUS
)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,NUM_UNIQUE_STATUS,NUM_MAX_STATUS
0,1803195,182943,-31,48.0,45.0,0,0,0,NaN,NaN
1,1715348,367990,-33,36.0,35.0,0,0,0,NaN,NaN
2,1784872,397406,-32,12.0,9.0,0,0,0,NaN,NaN
3,1903291,269225,-35,48.0,42.0,0,0,0,NaN,NaN
4,2341044,334279,-35,36.0,35.0,0,0,0,NaN,NaN


In [51]:
ignoredFields = ['TARGET']
# Assemble into I/O dataset format
#   X - All fields other than 'TARGET'
#   Y - 'TARGET' fields
dataCols = dataTrain.columns
trainDataMask = [col for col in dataCols if col not in ignoredFields]
x = dataTrain[trainDataMask]
y = dataTrain['TARGET']

# Summarize training dataset
featureCnt = len(dataTrain.keys()) - 1
numSamples = len(dataTrain)
print(f'Training dataset has {numSamples} samples, and {featureCnt} features')

# Split training data randomly using train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size = 0.10,
    random_state = randSeed
)

# Create lgb dataset
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_test = lgb.Dataset(data=x_test, label=y_test)

# Free up memory
del x
del y
del dataCols
del trainDataMask
del dataTrain
gc.collect()

Training dataset has 307511 samples, and 652 features


In [56]:
params = {
    'task': 'train',
    'device' : 'cpu',
    'nthread': 8,            # [CPU] number of OpenMP threads
    'gpu_use_dp' : 'false',  # [GPU] set to 1 to enable 64bit float point
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_iteration': 1000,
    'num_leaves': 32,
    'metric': 'auc',
    'reg_alpha': 5,
    'reg_lambda': 10,
    'learning_rate': 0.05,
    'max_bin': 256,
    'max_depth' : 10,
    'min_data_in_leaf': 32,
    'min_split_gain': 0.5,
    'min_child_weight': 1,
    'min_child_samples': 5,
    'subsample_for_bin': 200,
    'subsample': 1,
    'subsample_freq': 1,
    'colsample_bytree': 0.6,
    'verbose': 0
}

gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round = 50,
    valid_sets = lgb_test,
    early_stopping_rounds = 50,
    verbose_eval = 50
)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning:

Found `num_iteration` in params. Will use it instead of argument



Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.758439
[100]	valid_0's auc: 0.773542
[150]	valid_0's auc: 0.782224
[200]	valid_0's auc: 0.78665
[250]	valid_0's auc: 0.78883
[300]	valid_0's auc: 0.790031
[350]	valid_0's auc: 0.790758
[400]	valid_0's auc: 0.790981
[450]	valid_0's auc: 0.791342
[500]	valid_0's auc: 0.791474
[550]	valid_0's auc: 0.791472
Early stopping, best iteration is:
[532]	valid_0's auc: 0.791658


In [58]:
# Predict and save to csv
predResult = gbm.predict(dataTest)
submissionDataset = sample_submission
submissionDataset.TARGET = predResult
submissionDataset.to_csv('./lowpass_submission_v2.csv', index = False)